In [7]:
%%writefile aescore.py

import numpy as np
from random import randrange
from Crypto.Cipher import AES
np.set_printoptions(formatter={'int':hex})

PAIR = 0
BURST_PAIRS = 1
BURST_SINGLEKEY = 2
SEARCH_KEY = 3
BURST_OLDKEY = 4
IDENTIFIER = 0xAE
RETIDENTIFIER = 0xEA

# HEADER
START_BYTE = 0xAB

# Command
IDLE =  0x00
SEND =  0x01
RECIEVE =  0x04
RESET = 0x02
BUSY =  0x03


def burst_pairs_test(size = 8):
    keysize = 256
    datasize = 128
    base = 64
    nk = 0x8
    enc = 0x1
    mode = BURST_PAIRS
    #size = 8
    packetid = randrange(2**16)
    head = IDENTIFIER ^ enc << 8 ^ nk << 12 ^ mode << 16 ^ size << 24 ^ packetid << 48
    databuff = []
    cryptbuff = []
    databuff.append(head)
    for j in range(size):
        key = np.array([randrange(2**base) for i in range (keysize/base)], dtype=np.uint64)
        #keybuffer = np.getbuffer(key)
        data = np.array([randrange(2**base) for i in range (datasize/base)], dtype=np.uint64)
        #databuffer = np.getbuffer(data)
        
        mode = AES.MODE_ECB
        aes = AES.new(key, mode)
        encryptedbuffer = aes.encrypt(data)
        encrypted = np.frombuffer(encryptedbuffer, dtype=np.uint64)
        
        for i in key:
            databuff.append(i)
        for i in data:
            databuff.append(i)
        for i in encrypted:
            cryptbuff.append(i)
    #for i in range(16):
    #    databuff.append(0)
    
    #data = encapsulate_packet(from_index, node_index, databuff)
    databuff = np.array(databuff, dtype=np.uint64)
    cryptbuff = np.array(cryptbuff, dtype=np.uint64)
    return packetid, databuff, cryptbuff

def burst_pairs(key, data, size = 8):
    keysize = 256
    datasize = 128
    base = 64
    nk = 0x8
    enc = 0x1
    mode = BURST_PAIRS
    #size = 8
    packetid = randrange(2**16)
    head = IDENTIFIER ^ enc << 8 ^ nk << 12 ^ mode << 16 ^ size << 24 ^ packetid << 48
    databuff = []
    cryptbuff = []
    databuff.append(head)
    for j in range(size):
        
        mode = AES.MODE_ECB
        aes = AES.new(key, mode)
        encryptedbuffer = aes.encrypt(data)
        encrypted = np.frombuffer(encryptedbuffer, dtype=np.uint64)
        print encrypted
        
        for i in key:
            databuff.append(i)
        for i in data:
            databuff.append(i)
            
    #data = encapsulate_packet(from_index, node_index, databuff)
    databuff = np.array(databuff, dtype=np.uint64)
    return packetid, databuff

def burst_singlekey_test(size = 0x10):
    keysize = 256
    datasize = 128
    base = 64
    nk = 0x8
    enc = 0x1
    mode = BURST_SINGLEKEY
    #size = 8
    packetid = randrange(2**16)
    head = IDENTIFIER ^ enc << 8 ^ nk << 12 ^ mode << 16 ^ size << 24 ^ packetid << 48
    databuff = []
    cryptbuff = []
    databuff.append(head)
    key = np.array([randrange(2**base) for i in range (keysize/base)], dtype=np.uint64)
    #keybuffer = np.getbuffer(key)
    for i in key:
        databuff.append(i)
    #print key
    for j in range(size):
        data = np.array([randrange(2**base) for i in range (datasize/base)], dtype=np.uint64)
        #databuffer = np.getbuffer(data)
        
        mode = AES.MODE_ECB
        aes = AES.new(key, mode)
        encryptedbuffer = aes.encrypt(data)
        encrypted = np.frombuffer(encryptedbuffer, dtype=np.uint64)
        
        for i in data:
            databuff.append(i)
        for i in encrypted:
            cryptbuff.append(i)
    #for i in range(16):
    #    databuff.append(0)
        
    databuff = np.array(databuff, dtype=np.uint64)
    cryptbuff = np.array(cryptbuff, dtype=np.uint64)
    return packetid, databuff, cryptbuff

def burst_singlekey(key, data):
    size = len(data)/2
    keysize = 256
    datasize = 128
    base = 64
    nk = 0x8
    enc = 0x1
    mode = BURST_SINGLEKEY
    #size = 8
    packetid = randrange(2**16)
    head = np.array([IDENTIFIER ^ enc << 8 ^ nk << 12 ^ mode << 16 ^ size << 24 ^ packetid << 48], dtype=np.uint64)
    databuff1 = np.append(key, data)
    databuff = np.append(head, databuff1)
    #print databuff
    return packetid, databuff

def search_key(key, data, crypto, searchspace = 0):
    keysize = 256
    datasize = 128
    base = 64
    nk = 0x8
    enc = 0x1
    mode = SEARCH_KEY
    null = np.array([0x00, 0x00], dtype=np.uint64)
    #size = 8
    packetid = randrange(2**16)
    head = IDENTIFIER ^ enc << 8 ^ nk << 12 ^ mode << 16 ^ searchspace << 24 ^ packetid << 48
    databuff = []
    databuff.append(head)
    data = np.frombuffer(data, dtype=np.uint64)
    crypto = np.frombuffer(crypto, dtype=np.uint64)
    #print crypto
    #print '########'
        
    for i in key:
        databuff.append(i)
    for i in data:
        databuff.append(i)
    for i in crypto:
        databuff.append(i)
    
    databuff = np.array(databuff, dtype=np.uint64)    

    return packetid, databuff

Overwriting aescore.py


In [1]:
%%writefile partialkey.py
#%reload_ext autoreload
#%autoreload 2

import numpy as np
from Crypto.Cipher import AES
import aescore
import base64
import math
import cluster
import time
from multiprocessing import Process, Queue
import random
np.set_printoptions(formatter={'int':hex})

def createmask(searchptr, searchsize, val = 0):    
    keyindex = searchptr/64
    wordindex = searchptr%64
    mask = val << wordindex 
    #print 
    mask0 = mask & 0xFFFFFFFFFFFFFFFF
    mask1 = (mask >> 64) & 0xFFFFFFFFFFFFFFFF
    return mask0, mask1, keyindex


def blindmask(searchptr, searchsize):    
    keyindex = searchptr/64
    wordindex = searchptr%64
    mask = ((2**searchsize-1) << wordindex)
    mask0 = mask & 0xFFFFFFFFFFFFFFFF
    mask1 = (mask >> 64) & 0xFFFFFFFFFFFFFFFF
    
    return mask0 ^ 0xFFFFFFFFFFFFFFFF, mask1 ^ 0xFFFFFFFFFFFFFFFF, keyindex

class searchkey:
    def __init__(self):
        self.Cluster = cluster.cluster()
        self.Cluster.conf(3)
        self.nodes = [0x0001, 0x0002, 0x0004, 0x0005, 0x0006, 
                      0x0008, 0x0009, 0x0000a ] #, 0x000c, 0x000d, 0x000e, ] ##########
        #self.nodes_available = self.nodes
        
        self.node_struct = []
        for i in self.nodes:
            self.node_struct.append({'busy':0, 'watchdog': 0, 'group': 0, 'node': i})
            
            
    def reset(self):
        self.Cluster.reset()
        self.Cluster.conf(3)
        self.nodes = [0x0001, 0x0002, 0x0004, 0x0005, 0x0006, 
                      0x0008, 0x0009, 0x0000a] #, 0x000c, 0x000d, 0x000e, ] ##########
        
        self.node_struct = []
        for i in self.nodes:
            self.node_struct.append({'busy':0, 'watchdog': 0, 'group': 0, 'node': i})
            
    def search(self, key, data, crypto, queue, searchptr = 0, searchsize = 0):
        #print key
        WATCHDOG = 7
        STUCK_WATCHDOG = 10
        NODESEARCHSPACE = 26
        
        self.key = key
        self.crypto = base64.b64decode(crypto)
        self.data = data
        self.searchptr = searchptr
        self.searchsize = searchsize
        
        #self.packets = []
        keyindex = searchptr/64
        wordindex = searchptr%64
        reskey = 0
        
        for i in self.node_struct:
            i['busy'] = 0
            i['watchdog'] = 0
            i['group'] = 0
            i['packet'] = {}
            
        if searchsize > NODESEARCHSPACE:
            groupsize = searchsize - NODESEARCHSPACE
            newsearchptr = groupsize + searchptr            
        else:
            groupsize = 0
            newsearchptr = searchptr
            
        self.group = range(2**groupsize)
        GLOBALTIMEOUT = 2**groupsize * 3
        globalwatchdog = 0
        
        while 1: #len(self.group) or len(self.packets):
            try:
                busy_node = next(index for (index, d) in enumerate(self.node_struct) if d['busy'] == 1)
            except:
                if not len(self.group):
                    break
                pass
            try:
                freenode = next(index for (index, d) in enumerate(self.node_struct) if d['busy'] == 0)                
                time.sleep(0.1)
                #print freenode
                #print '#####', self.node_struct[freenode]
                #stuckwatchdog = 0 #STUCK_WATCHDOG = 0
                newkey = np.copy(self.key)  #.copy()
                node = self.node_struct[freenode]['node']
                queue.put({'group':self.group})
                
                val = self.group.pop(random.randrange(len(self.group)))
                mask0, mask1, keyindex = createmask(searchptr, searchsize, val = val)
                mask2, mask3, keyindex = blindmask(searchptr, searchsize)
                newkey[keyindex] = (int(newkey[keyindex]) & mask2) ^ mask0
                newkey[keyindex+1] = (int(newkey[keyindex+1]) & mask3) ^ mask1
                packetid, databuff = aescore.search_key(newkey, self.data, self.crypto, searchspace = newsearchptr)
                self.node_struct[freenode]['packet'] = {'packetid':packetid, 
                                                        'packet':np.copy(databuff), 
                                                        'key':np.copy(newkey), 
                                                        'searchptr':newsearchptr,
                                                        'group':val}
                
                self.node_struct[freenode]['group'] = val
                self.node_struct[freenode]['busy'] = 1
                #print '#####', self.node_struct[freenode]
                self.Cluster.sendpackets(databuff, source=0x0000, sink=node)
                
            except:   
                time.sleep(1)     
                #print '##################################'
                #print 'group          :', self.group 
                '''for i in self.node_struct:                
                    print i['node'], i['busy'], i['watchdog'], i['group'],
                    try:
                        print i['packet']['packetid']
                    except:
                        print '---'
                        pass'''
                    
                for idx, val in enumerate(self.node_struct): #for i in self.node_struct:
                    if val['busy'] == 1:
                        if val['watchdog'] >= WATCHDOG:
                            print ',', val['node'],
                            self.group.append(val['group'])
                            
                            self.node_struct[idx]['busy'] = 0
                            self.node_struct[idx]['watchdog'] = 0
                            self.node_struct[idx]['group'] = 0
                            self.node_struct[idx]['packet'] = None
                        else:                            
                            self.node_struct[idx]['watchdog'] = self.node_struct[idx]['watchdog'] + 1                        
                
                globalwatchdog = globalwatchdog + 1
                #print 'GLOBALTIMEOUT' , globalwatchdog
                if globalwatchdog == GLOBALTIMEOUT:
                    return 0, 0
                
                res, reskey = self.getresult()
                #print '####', res
                if res:
                    return res, reskey
        return 0, reskey
            
     
    def getresult(self):
        ids = []
        retids = []
        keys = []
        for i in self.node_struct:
            try:
                ids.append(i['packet']['packetid'])
            except:
                pass
            pass
            
        #print '@@@@>>', ids
        retbuff = self.Cluster.getpackets(ids)
        #print '@@@@>>',retbuff
        if len(retbuff):
            #print '<<', retbuff
            for i in retbuff:
                retids.append(i['packetid'])
            #print '@@@@>>', retids
            
            #print '%%%%%%%%%%%%%%%%'
            #for i in self.node_struct:
            #    #print '$', i['packet']['packetid'],
            for i in retbuff:
                #print '&&&&#', i['packetid']
                #print '#####', self.node_struct
                
                #print '###$$', next(index for (index, d) in enumerate(self.node_struct) 
                #                    if d['packet']['packetid'] == i['packetid'])
                
                try:
                    #print self.node_struct
                    for index, d in enumerate(self.node_struct):
                        try:
                            if d['packet']['packetid'] == i['packetid']:
                                respnode = index
                        except:
                            #print 'Packet not present for node ', index
                            pass
                            
                    #respnode = next(index for (index, d) in enumerate(self.node_struct) 
                    #                if d['packet']['packetid'] == i['packetid'])
                    #print '&&&&', i['packetid']
                    
                    if ((int(i['packet'][0]) & 0x000000FFFF000000) >> 24) == 0xFFFF:
                        searchctr = int(i['packet'][2])
                        print self.node_struct[respnode] 
                        response = self.backsearch(self.node_struct[respnode]['packet']['key'], 
                                                   searchctr, 
                                                   searchptr = self.node_struct[respnode]['packet']['searchptr'])
                        print "Sollution found"
                        #print response
                        return response
                    else:
                        response = (0, 0)
                    self.node_struct[respnode]['busy'] = 0
                    self.node_struct[respnode]['watchdog'] = 0
                    self.node_struct[respnode]['group'] = 0
                    self.node_struct[respnode]['packet'] = None
                except:
                    print 'error !@#!@$!$'
                    print self.node_struct
                #print '@@@@@'
                
        return (0, 0)
        
    def getstatus(self):
        return self.group
    
    def backsearch(self, key, searchctr, searchptr = 0):
        #print 'backsearch'
        data = self.data
        crypto = self.crypto
        #print data
        #print crypto
        offset = 0x00
        mode = AES.MODE_ECB
        
        for i in range(64):
            val = searchctr - i - offset
            if val <= 0:
                val = 2**6 -1 - val
            newkey = np.copy(key)
            mask0, mask1, keyindex = createmask(searchptr, 26, val = val)
            mask2, mask3, keyindex = blindmask(searchptr, 26)
            newkey[keyindex] = (int(newkey[keyindex]) & mask2) ^ mask0
            newkey[keyindex+1] = (int(newkey[keyindex+1]) & mask3) ^ mask1
            #print hex(newkey[keyindex])
            #print hex(newkey[keyindex+1])
                        
            aes = AES.new(newkey, mode)
            aes.use_aesni=True
            newcrypto = aes.encrypt(data)
            #print base64.b64encode(crypto)
            #print base64.b64encode(newcrypto)
            if crypto == newcrypto:
                #print newkey
                return (1, newkey)
        return (0, 0)      



Overwriting partialkey.py


In [4]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from Crypto.Cipher import AES
import aescore
import base64
import math
import cluster
import time
from multiprocessing import Process, Queue
import random

def createmask(searchptr, searchsize, val = 0):    
    keyindex = searchptr/64
    wordindex = searchptr%64
    mask = ((2**searchsize-1 ^ val) << wordindex) 
    mask0 = mask & 0xFFFFFFFFFFFFFFFF
    mask1 = (mask >> 64) & 0xFFFFFFFFFFFFFFFF
    return mask0, mask1, keyindex

class searchkey:
    def __init__(self):
        self.Cluster = cluster.cluster()
        self.Cluster.conf(4)
        self.nodes = [0x0001, 0x0002, 0x0004, 0x0005, 0x0006, 
                      0x0008, 0x0009, 0x00010, 0x000c, 0x000d, 0x000e, ] ##########
        #self.nodes_available = self.nodes
        
        self.node_struct = []
        for i in self.nodes:
            self.node_struct.append({'busy':0, 'watchdog': 0, 'group': 0, 'node': i})
            
            
    def reset(self):
        self.Cluster.reset()
        self.Cluster.conf(4)
        self.nodes = [0x0001, 0x0002, 0x0004, 0x0005, 0x0006, 
                      0x0008, 0x0009, 0x00010, 0x000c, 0x000d, 0x000e, ] ##########
        
        self.node_struct = []
        for i in self.nodes:
            self.node_struct.append({'busy':0, 'watchdog': 0, 'group': 0, 'node': i})
            
    def search(self, key, data, crypto, queue, searchptr = 0, searchsize = 0):
        #print key
        WATCHDOG = 10
        STUCK_WATCHDOG = 10
        NODESEARCHSPACE = 26
        
        self.key = key
        self.crypto = base64.b64decode(crypto)
        self.data = data
        self.searchptr = searchptr
        self.searchsize = searchsize
        
        #self.packets = []
        keyindex = searchptr/64
        wordindex = searchptr%64
        reskey = 0
        
        for i in self.node_struct:
            i['busy'] = 0
            i['watchdog'] = 0
            i['group'] = 0
            i['packet'] = {}
            
        if searchsize > NODESEARCHSPACE:
            groupsize = searchsize - NODESEARCHSPACE
            newsearchptr = groupsize + searchptr            
        else:
            groupsize = 0
            newsearchptr = searchptr
            
        self.group = range(2**groupsize)
        GLOBALTIMEOUT = 2**groupsize * 2
        globalwatchdog = 0
        
        while 1: #len(self.group) or len(self.packets):
            print 'node_struct', self.node_struct
            #print next(index for (index, d) in enumerate(self.node_struct) if d['busy'] == 0)
            try:
                busy_node = next(index for (index, d) in enumerate(self.node_struct) if d['busy'] == 1)
                print 'busy_node', busy_node
            except:
                if not len(self.group):
                    break
                pass
            try:
                freenode = next(index for (index, d) in enumerate(self.node_struct) if d['busy'] == 0)
                print '$$$$$$$$$$$$$$$$$$$$$$$$'
                print 'freenode', freenode
                print '$$$$$$$$$$$$$$$$$$$$$$$$1'
                #print self.node_struct[freenode]
                #stuckwatchdog = 0 #STUCK_WATCHDOG = 0
                print '$$$$$$$$$$$$$$$$$$$$$$$$3'
                print 'key', key
                print 'key', self.key
                newkey = self.key[:]
                print '$$$$$$$$$$$$$$$$$$$$$$$$2'
                node = self.node_struct[freenode]['node']
                #queue.put({'group':self.group})
                
                val = self.group.pop(random.randrange(len(self.group)))
                print 'val', val
                mask0, mask1, keyindex = createmask(searchptr, searchsize, val = val)
                mask2, mask3, keyindex = createmask(searchptr, searchsize)
                newkey[keyindex] = int(newkey[keyindex]) ^ mask2 ^ mask0
                newkey[keyindex+1] = int(newkey[keyindex+1]) ^ mask3 ^ mask1
                packetid, databuff = aescore.search_key(newkey, self.data, self.crypto, searchspace = newsearchptr)                
                print 'packetid', packetid
                print 'databuff', databuff
                self.node_struct[freenode]['packet'] = {'packetid':packetid, 
                                                        'packet':databuff, 
                                                        'key':newkey, 
                                                        'searchptr':newsearchptr,
                                                        'group':val}
                
                self.node_struct[freenode]['group'] = val
                self.node_struct[freenode]['busy'] = 1
                self.Cluster.sendpackets(databuff, source=0x0000, sink=node)
                
            except:   
                time.sleep(1)     
                print '##################################'
                print 'group          :', self.group 
                for i in self.node_struct:                
                    print i['node'], i['busy'], i['watchdog'], i['group']
                for idx, val in enumerate(self.node_struct): #for i in self.node_struct:
                    if val['busy'] == 1:
                        if val['watchdog'] >= WATCHDOG:
                            print 'WATCHDOG expired for :', val
                            self.group.append(val['group'])
                            
                            self.node_struct[idx]['busy'] = 0
                            self.node_struct[idx]['watchdog'] = 0
                            self.node_struct[idx]['group'] = 0
                            self.node_struct[idx]['packet'] = None
                        else:                            
                            self.node_struct[idx]['watchdog'] = self.node_struct[idx]['watchdog'] + 1                        
                
                globalwatchdog = globalwatchdog + 1
                #print 'GLOBALTIMEOUT' , globalwatchdog
                if globalwatchdog == GLOBALTIMEOUT:
                    return 0, 0
                
                res, reskey = self.getresult()
                print '####', res
                if res:
                    return res, reskey
        return 0, reskey
            
     
    def getresult(self):
        ids = []
        for i in self.node_struct:
            try:
                ids.append(i['packet']['packetid'])
            except:
                pass
            
        #print '@@@@!>>', ids
        retbuff = self.Cluster.getpackets(ids)
        #print '@@@@!>>',retbuff
        if len(retbuff):
            for i in retbuff:
                try:
                    respnode = next(index for (index, d) in enumerate(self.node_struct) if 
                                    d['packet']['packetid'] == i['packetid'])
                    print packetid, 'matches'
                    if ((int(i['packet'][0]) & 0x000000FFFF000000) >> 24) == 0xFFFF:
                        searchctr = int(i['packet'][2]) 
                        response = self.backsearch(self.node_struct[respnode]['packet']['key'], 
                                                   searchctr, 
                                                   searchptr = self.node_struct[respnode]['packet']['searchptr'])
                    else:
                        response = (0, 0)
                    self.node_struct[respnode]['busy'] = 0
                    self.node_struct[respnode]['watchdog'] = 0
                    self.node_struct[respnode]['group'] = 0
                    self.node_struct[respnode]['packet'] = None

                    return response
                    
                except:
                    pass
                
        return (0, 0)
        
    def getstatus(self):
        return self.group
    
    def backsearch(self, key, searchctr, searchptr = 0):
        data = self.data
        crypto = self.crypto
        #print data
        #print crypto
        offset = 0x00
        mode = AES.MODE_ECB
        #print 'backsearch:', searchctr
        #print 'backsearch:', searchptr
        #print key
        for i in range(64):
            val = searchctr - i - offset
            if val <= 0:
                val = 2**26 -1 - val
            newkey = key.copy()
            mask0, mask1, keyindex = createmask(searchptr, 26, val = val)
            #print hex(mask0), hex(mask1), hex(keyindex)
            mask2, mask3, keyindex = createmask(searchptr, 26)
            #print hex(mask2), hex(mask3), hex(keyindex)
            newkey[keyindex] = int(newkey[keyindex]) ^ mask2 ^ mask0
            newkey[keyindex+1] = int(newkey[keyindex+1]) ^ mask3 ^ mask1
            #print hex(mask0)
            #print 'backsearch:', newkey
            
            aes = AES.new(newkey, mode)
            aes.use_aesni=True
            newcrypto = aes.encrypt(data)
            #print base64.b64encode(crypto)
            #print base64.b64encode(newcrypto)
            if crypto == newcrypto:
                #print newkey
                return (1, newkey)
        return (0, 0)      

    
Searchkey = searchkey()
Searchkey.reset()
key=[0x98fc827cc0000000L, 0xaf2f0a96dea3fff9L, 0x36a8532728212eb1L,
 0x6954cdcea27cc32fL]
data='Abhishek Bajpai.'
crypto='TMNOa27nq+qAaQ9VD8PKKA=='
searchptr=0
searchsize=30
queue = Queue()
Searchkey.search(key, data, crypto, queue, searchptr = searchptr, searchsize = searchsize)

PCI is being Initialized....
PCI Initialized
BAR0 Size: 0x10000
BAR1 Size: 0x100000
pci_status : 0xffffffffffd705feL
pci_command : 0xffffffff000000feL
pci_status : 0xffd70500L
pci_command : 0x100100L
setting node 0 with address : 0
setting node 1 with address : 4
setting node 2 with address : 8
setting node 3 with address : 12
setting node 0 with address : 0
setting node 1 with address : 4
setting node 2 with address : 8
setting node 3 with address : 12
node_struct [{'node': 1, 'watchdog': 0, 'busy': 0, 'group': 0, 'packet': {}}, {'node': 2, 'watchdog': 0, 'busy': 0, 'group': 0, 'packet': {}}, {'node': 4, 'watchdog': 0, 'busy': 0, 'group': 0, 'packet': {}}, {'node': 5, 'watchdog': 0, 'busy': 0, 'group': 0, 'packet': {}}, {'node': 6, 'watchdog': 0, 'busy': 0, 'group': 0, 'packet': {}}, {'node': 8, 'watchdog': 0, 'busy': 0, 'group': 0, 'packet': {}}, {'node': 9, 'watchdog': 0, 'busy': 0, 'group': 0, 'packet': {}}, {'node': 16, 'watchdog': 0, 'busy': 0, 'group': 0, 'packet': {}}, {'node':

(0, 0)

In [13]:
import random
print random.randrange(100)

43


In [15]:
a = [{'s':0, 'b':3, 'p':{'a':1}},
    {'s':0, 'b':3, 'p':{'a':1}},
    {'s':0, 'b':3, 'p':{'a':1}},
    {'s':0, 'b':3, 'p':None},]
print a

[{'p': {'a': 1}, 's': 0, 'b': 3}, {'p': {'a': 1}, 's': 0, 'b': 3}, {'p': {'a': 1}, 's': 0, 'b': 3}, {'p': None, 's': 0, 'b': 3}]


In [21]:
ids = []
for i in a:
    try:
        ids.append(i['p']['a'])
    except:
        pass
print ids

[1, 1, 1]
